# Pós-Graduação - Ciência de Dados & Big Data

## Pontifícia Universidade Católica de Minas Gerais (PUC-MG)

### Aluno: Victor Hugo Negrisoli

### Utilizando funções Spark: operações com RDD

In [30]:
from pyspark.sql import SparkSession
import json

Neste Notebook serão utilizadas funções do Spark trabalhando com RDDs, são elas:

* Map
* Reduce
* Filter
* Collect
* Count
* CountByValue
* Parallelize
* Aggregate
* Union
* TakeSample
* FlatMap

### Importando dados a partir de um dicionário

In [2]:
pessoas = []
pessoas.append({
  'id': 1,
  'nome': 'Bob',
  'idade': 45,
  'gen': 'M'
})
pessoas.append({
  'id': 2,
  'nome': 'Gloria',
  'idade': 43,
  'gen': 'F'
})
pessoas.append({
  'id': 4,
  'nome': 'Albert',
  'idade': 28,
  'gen': 'M'
})
pessoas.append({
  'id': 5,
  'nome': 'Laura',
  'idade': 33,
  'gen': 'F'
})
pessoas.append({
  'id': 8,
  'nome': 'Simone',
  'idade': 18,
  'gen': 'T'
})
pessoas.append({
  'id': 12,
  'nome': 'Marta',
  'idade': 45,
  'gen': 'F'
})
pessoas.append({
  'id': 45,
  'nome': 'Jairo',
  'idade': 82,
  'gen': 'M'
})
pessoas.append({
  'id': 13,
  'nome': 'Teste',
  'idade': 38,
  'gen': 'T'
})

In [3]:
spark = SparkSession\
            .builder\
            .appName("Pratica - Funções Spark")\
            .getOrCreate()

In [4]:
sc = spark.sparkContext

In [5]:
pessoasRDD = sc.parallelize(pessoas)

In [6]:
lista_collect = pessoasRDD.collect()
lista_collect

[{'id': 1, 'nome': 'Bob', 'idade': 45, 'gen': 'M'},
 {'id': 2, 'nome': 'Gloria', 'idade': 43, 'gen': 'F'},
 {'id': 4, 'nome': 'Albert', 'idade': 28, 'gen': 'M'},
 {'id': 5, 'nome': 'Laura', 'idade': 33, 'gen': 'F'},
 {'id': 8, 'nome': 'Simone', 'idade': 18, 'gen': 'T'},
 {'id': 12, 'nome': 'Marta', 'idade': 45, 'gen': 'F'},
 {'id': 45, 'nome': 'Jairo', 'idade': 82, 'gen': 'M'},
 {'id': 13, 'nome': 'Teste', 'idade': 38, 'gen': 'T'}]

Filtarndo pessoas acima de 30 anos com a função ``Filter``

In [7]:
lista_maiores_30 = pessoasRDD \
                        .filter(lambda x: x['idade'] > 30)
lista_maiores_30.collect()

[{'id': 1, 'nome': 'Bob', 'idade': 45, 'gen': 'M'},
 {'id': 2, 'nome': 'Gloria', 'idade': 43, 'gen': 'F'},
 {'id': 5, 'nome': 'Laura', 'idade': 33, 'gen': 'F'},
 {'id': 12, 'nome': 'Marta', 'idade': 45, 'gen': 'F'},
 {'id': 45, 'nome': 'Jairo', 'idade': 82, 'gen': 'M'},
 {'id': 13, 'nome': 'Teste', 'idade': 38, 'gen': 'T'}]

In [8]:
total_acima_30 = lista_maiores_30.count()
total_acima_30

6

Verificando todos os registros que fazem parte do sexo masculino

In [9]:
lista_masculinos = pessoasRDD \
                        .filter(lambda x: x['gen'].upper() == 'M')
lista_masculinos.collect()

[{'id': 1, 'nome': 'Bob', 'idade': 45, 'gen': 'M'},
 {'id': 4, 'nome': 'Albert', 'idade': 28, 'gen': 'M'},
 {'id': 45, 'nome': 'Jairo', 'idade': 82, 'gen': 'M'}]

In [10]:
lista_masculinos.count()

3

Visualizando o total de pessoas por gênero diferente de T

In [16]:
pessoas_por_genero = pessoasRDD \
                        .filter(lambda gen: gen['gen'] != 'T') \
                        .map(lambda gen: gen['gen'])

pessoas_por_genero.countByValue()

defaultdict(int, {'M': 3, 'F': 3})

Visualizando a média de idade das pessoas com a função `Aggregate`

In [14]:
sequencia_op = (lambda x, y: (x[0] + y['idade'], x[1] + 1))
combinacao_op = (lambda x, y: (x[0] + y[0], x[1] + y[1]))

[soma, quantidade] = pessoasRDD \
                        .aggregate((0, 0), sequencia_op, combinacao_op)
media = soma / quantidade
media

41.5

Realizando uma união entre as pessoas acima de 30 com as pessoas do sexo Masculino com a função `Union`

In [29]:
lista_masculinos_e_maiores_30 = lista_masculinos \
                                    .union(lista_maiores_30)
lista_masculinos_e_maiores_30.collect()

[{'id': 1, 'nome': 'Bob', 'idade': 45, 'gen': 'M'},
 {'id': 4, 'nome': 'Albert', 'idade': 28, 'gen': 'M'},
 {'id': 45, 'nome': 'Jairo', 'idade': 82, 'gen': 'M'},
 {'id': 1, 'nome': 'Bob', 'idade': 45, 'gen': 'M'},
 {'id': 2, 'nome': 'Gloria', 'idade': 43, 'gen': 'F'},
 {'id': 5, 'nome': 'Laura', 'idade': 33, 'gen': 'F'},
 {'id': 12, 'nome': 'Marta', 'idade': 45, 'gen': 'F'},
 {'id': 45, 'nome': 'Jairo', 'idade': 82, 'gen': 'M'},
 {'id': 13, 'nome': 'Teste', 'idade': 38, 'gen': 'T'}]

Recuperando valores aleatórios de uma amostragem de 3 valores

In [31]:
tres_aleatorios = lista_masculinos_e_maiores_30 \
                                                .takeSample(True, 3)
tres_aleatorios.collect()

[{'id': 12, 'nome': 'Marta', 'idade': 45, 'gen': 'F'},
 {'id': 45, 'nome': 'Jairo', 'idade': 82, 'gen': 'M'},
 {'id': 13, 'nome': 'Teste', 'idade': 38, 'gen': 'T'}]

### Importando dados de um arquivo JSON

In [34]:
pessoas_json = sc.textFile('pessoasval.json')
pessoas_json.collect()

['[{"id": 1, "val": 45, "dat":"12/01/2006"}, {"id": 2, "val": 53, "dat":"04/06/2009"}, {"id": 4, "val": 345, "dat":"18/01/2012"}, {"id": 5, "val": 435, "dat":"12/01/2016"}, {"id": 8, "val": 2003, "dat":"08/09/2015"}, {"id": 12, "val": 100, "dat":"12/11/2000"}, {"id": 45, "val": 200, "dat":"12/01/2006"},{"id": 13, "val": 99999, "dat":"12/01/2006"},{"id": 1, "val": 405, "dat":"12/03/2006"}, {"id": 2, "val": 503, "dat":"04/09/2009"}, {"id": 4, "val": 35, "dat":"01/10/2012"}, {"id": 5, "val": 45, "dat":"12/12/2016"}, {"id": 8, "val": 23, "dat":"01/01/2015"}, {"id": 12, "val": 10, "dat":"02/01/2002"}, {"id": 45, "val": 20, "dat":"12/12/2006"},{"id": 13, "val": 99999, "dat":"12/01/2007"}]']

Criando a lista de pessoas do RDD com a função `flatMap`

In [36]:
pessoasRDD = pessoas_json \
                        .flatMap(lambda x: json.loads(x))
pessoasRDD.collect()

[{'id': 1, 'val': 45, 'dat': '12/01/2006'},
 {'id': 2, 'val': 53, 'dat': '04/06/2009'},
 {'id': 4, 'val': 345, 'dat': '18/01/2012'},
 {'id': 5, 'val': 435, 'dat': '12/01/2016'},
 {'id': 8, 'val': 2003, 'dat': '08/09/2015'},
 {'id': 12, 'val': 100, 'dat': '12/11/2000'},
 {'id': 45, 'val': 200, 'dat': '12/01/2006'},
 {'id': 13, 'val': 99999, 'dat': '12/01/2006'},
 {'id': 1, 'val': 405, 'dat': '12/03/2006'},
 {'id': 2, 'val': 503, 'dat': '04/09/2009'},
 {'id': 4, 'val': 35, 'dat': '01/10/2012'},
 {'id': 5, 'val': 45, 'dat': '12/12/2016'},
 {'id': 8, 'val': 23, 'dat': '01/01/2015'},
 {'id': 12, 'val': 10, 'dat': '02/01/2002'},
 {'id': 45, 'val': 20, 'dat': '12/12/2006'},
 {'id': 13, 'val': 99999, 'dat': '12/01/2007'}]

Verificando a quantidade de IDs de cada pessoa existente no RDD

In [37]:
pessoas_ids = pessoasRDD \
                        .map(lambda x: x['id'])
pessoas_ids.countByValue()

defaultdict(int, {1: 2, 2: 2, 4: 2, 5: 2, 8: 2, 12: 2, 45: 2, 13: 2})